Implement a barrier method for solving:

$\min \frac{1}{2}x^TQx + b^Tx$

subject to $x \geq 0$.


In [1]:
#pip install numpy


In [2]:
import numpy as np

In [3]:
#Define the objective function

def original_f(x, Q, b):
    return 0.5 * x.T @ Q @ x + b.T @ x


In [4]:
def barrier_f(x, Q, b, t):
    return t*(0.5 * x.T @ Q @ x + b.T @ x) - np.sum(np.log(x)) 

In [5]:
#Define the gradient of the objective function

def gradient_barrier_f(x, Q, b, t):
    return t*(Q @ x + b) - 1/x


In [6]:
#Define the Hessian of the objective function

def hessian_barrier_f(x, Q, b, t):
    n = len(x)
    H = np.zeros((n, n))
    for i in range(n):
        H[i, i] = 1 / x[i]**2
    return t*Q + H


In [7]:
#Define a function that can guarantee a vector with at least one negative component

def generate_vector_with_negative(size, mean=0, std=1):
    # Generate a vector from normal distribution
    vector = np.random.normal(mean, std, size)
    
    # Check if all elements are positive
    if np.all(vector >= 0):
        # Choose a random index and negate the corresponding element
        random_index = np.random.randint(0, size)
        vector[random_index] *= -1
    
    # Reshape the vector to be a column vector
    vector = vector.reshape(-1, 1)
    
    return vector


In [8]:
#Define function for barrier method

def barrier_method(x_init, Q, b, t, mu, inner_tol = 1e-10, outer_tol = 1e-10, max_iter = 100):
    
    x = x_init
    alpha = 0.2
    beta = 0.5
    m = len(x)
    
    for __ in range(max_iter):
        for _ in range(max_iter): #starting Newton's method
            grad = gradient_barrier_f(x, Q, b, t) #calculate gradient of the barrier function
            hess = hessian_barrier_f(x, Q, b, t) #calculate hessian of the barrier function
            newton_step = -np.linalg.solve(hess, grad) #calculate newton step
            newton_step_norm = np.linalg.norm(newton_step)
            stop_criterion = (newton_step_norm ** 2)/2 #calculate stop criterion, a.k.a lambda squared divided by two
            print("Inner decrement: ", stop_criterion)
            if stop_criterion <= inner_tol: #exit newtons method if the stop criterion is smaller than our tolerance level 
                break
            
            #if we the norm of the newton step isn't sufficiently small we iterate with step length t
            #making sure no component exit the dominion of x by adjusting step length t
            step_length = 1.0 #initial step length
            while True:
                x_new = x + step_length*newton_step #calculate new x
                if np.all(x_new > 0): #if all components are larger than zero we perform backtracking line search with alpha and beta
                    while barrier_f(x_new, Q, b, t) > barrier_f(x, Q, b, t) + alpha*step_length*grad.T @ newton_step: #perform back tracking line search for as long as the new x does not give us a better function value
                        step_length *= beta 
                        x_new = x + step_length*newton_step 
                    x = x_new 
                    break 
                else: #if the new x has components that are not larger than zero we decrease t
                    step_length /= 2
        print("Outer decrement: ", m/t)         
        if m/t < outer_tol: #exit outer loop if m/t is less than our tolerance level
            break
        else:
            t *= mu #else we increase t and start Newton's method over again
    return x
        

In [9]:
#Define the conditions for your problem

#size of problem
n = 5

#decide parameters
mu = 15
t = 1

#initial point
x0 = np.ones((n, 1))

#generate a symmetric positive definite matrix Q
B = np.random.rand(n, n)
Q = B.T @ B

#generate vector b with at least one negative element
b = generate_vector_with_negative(len(x0))


In [10]:
#Solve the problem using a barrier method

x_optimal = barrier_method(x0, Q, b, t, mu)

print("The vector x that minimizes the objective function is \n", x_optimal)
print("The function value of that vector x is equal to \n", original_f(x_optimal, Q, b))



Inner decrement:  5.803426190969354
Inner decrement:  1.90424520701686
Inner decrement:  0.1565110600453717
Inner decrement:  0.0064781417410287555
Inner decrement:  0.0007386185359777351
Inner decrement:  5.269716162445579e-06
Inner decrement:  2.071227830622304e-10
Inner decrement:  3.097532820155473e-19
Outer decrement:  5.0
Inner decrement:  12.580510716651876
Inner decrement:  2.746765217338432
Inner decrement:  0.8180233436369978
Inner decrement:  0.016907478264781922
Inner decrement:  0.012107755787076594
Inner decrement:  3.7621621339633335e-06
Inner decrement:  4.0437766879843865e-09
Inner decrement:  6.4754604352955196e-15
Outer decrement:  0.3333333333333333
Inner decrement:  0.6582013217603798
Inner decrement:  0.03306300954671422
Inner decrement:  8.55933497099539e-05
Inner decrement:  7.648825351285191e-05
Inner decrement:  3.825961221358456e-07
Inner decrement:  3.429286471545989e-08
Inner decrement:  1.8549472501177614e-10
Inner decrement:  5.4904375486616444e-15
Outer 